# Final Exam
Submitted by: Anthony Aromin

## Initial Setup

In [343]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [344]:
df = pd.read_csv('salary_dataset.csv')
df.head(5)

,Position,Annual Salary in Dollar,Date Hired
0,City Administrator,295000.0,2004.0
1,President,295000.0,2008.0
2,Superintendent Of Schools,275000.0,2007.0
3,Chief,230743.0,1990.0
4,Medical Officer Psych,206000.0,2008.0


In [345]:
df.columns = [
    'position',
    'annual_salary_usd',
    'date_hired'
]
df.head(5)

,position,annual_salary_usd,date_hired
0,City Administrator,295000.0,2004.0
1,President,295000.0,2008.0
2,Superintendent Of Schools,275000.0,2007.0
3,Chief,230743.0,1990.0
4,Medical Officer Psych,206000.0,2008.0


## Data Cleaning

In [346]:
df.dropna(subset=[
    'position',
    'date_hired'
],inplace=True)

df.isnull().sum()

position              0
annual_salary_usd    64
date_hired            0
dtype: int64

In [347]:
df.isnull().sum()

position              0
annual_salary_usd    64
date_hired            0
dtype: int64

In [348]:
df.describe()

,annual_salary_usd,date_hired
count,11434.000000,11498.000000
mean,88106.521077,1996.428596
std,25158.378394,10.094801
min,7.000000,1955.000000
25%,78743.000000,1988.000000
50%,85606.000000,1998.000000
75%,97296.000000,2006.000000
max,295000.000000,2011.000000


In [349]:
df['date_hired'] = df['date_hired'].astype(int)

df.head(5)

,position,annual_salary_usd,date_hired
0,City Administrator,295000.0,2004
1,President,295000.0,2008
2,Superintendent Of Schools,275000.0,2007
3,Chief,230743.0,1990
4,Medical Officer Psych,206000.0,2008


In [350]:
temp_df = df
temp_grouped_df = df.groupby('position')['annual_salary_usd']

def fill_annual_salary_usd(x):
    if (np.isnan(x['annual_salary_usd']) or
        x['annual_salary_usd'] < 10_000.00):
        mean_salary = temp_grouped_df.get_group(x['position']).mean()

        # if mean salary of position is still NaN or way less than minimum wage
        # use mean salary of all positions instead.
        if (np.isnan(mean_salary) or
            mean_salary < 10_000.00):
            mean_salary = df['annual_salary_usd'].mean()

        return mean_salary
    else:
        return x['annual_salary_usd']

df['annual_salary_usd'] = temp_df.apply(fill_annual_salary_usd, axis=1)

print(df.isnull().sum())
print('-' * 50)
print(df.describe())

position             0
annual_salary_usd    0
date_hired           0
dtype: int64
--------------------------------------------------
       annual_salary_usd    date_hired
count       11498.000000  11498.000000
mean        91586.769790   1996.428596
std         18200.788921     10.094801
min         71796.000000   1955.000000
25%         78743.000000   1988.000000
50%         87987.000000   1998.000000
75%         97674.250000   2006.000000
max        295000.000000   2011.000000


## Additional Columns

In [351]:
df['annual_salary_php'] = df.apply(
    lambda x: x['annual_salary_usd'] * 50.0, axis=1
)

df.head(5)

,position,annual_salary_usd,date_hired,annual_salary_php
0,City Administrator,295000.0,2004,14750000.0
1,President,295000.0,2008,14750000.0
2,Superintendent Of Schools,275000.0,2007,13750000.0
3,Chief,230743.0,1990,11537150.0
4,Medical Officer Psych,206000.0,2008,10300000.0


In [352]:
df['monthly_salary_php'] = df.apply(
    lambda x: np.round(x['annual_salary_php'] / 12.0, 2), axis=1
)

df.head(5)

,position,annual_salary_usd,date_hired,annual_salary_php,monthly_salary_php
0,City Administrator,295000.0,2004,14750000.0,1229166.67
1,President,295000.0,2008,14750000.0,1229166.67
2,Superintendent Of Schools,275000.0,2007,13750000.0,1145833.33
3,Chief,230743.0,1990,11537150.0,961429.17
4,Medical Officer Psych,206000.0,2008,10300000.0,858333.33


In [353]:
df['years_exp'] = df.apply(
    lambda x: 2015 - x['date_hired'], axis = 1
)

df.head(5)

,position,annual_salary_usd,date_hired,annual_salary_php,monthly_salary_php,years_exp
0,City Administrator,295000.0,2004,14750000.0,1229166.67,11
1,President,295000.0,2008,14750000.0,1229166.67,7
2,Superintendent Of Schools,275000.0,2007,13750000.0,1145833.33,8
3,Chief,230743.0,1990,11537150.0,961429.17,25
4,Medical Officer Psych,206000.0,2008,10300000.0,858333.33,7


## Exporting the Dataset

##  
---

## Training